In [1]:
%matplotlib inline

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng

from mesa import Agent, Model

'''
Time in most agent-based models moves in steps, sometimes also called ticks. At each step of the model, one or more of the agents – usually all of them – are activated and take their own step, changing internally and/or interacting with one another or the environment.

The scheduler is a special model component which controls the order in which agents are activated.
The RandomActivation scheduler activates all the agents once per step, in random order.
'''
from mesa.time import RandomActivation

'''
Many ABMs have a spatial element, with agents moving around and interacting with nearby neighbors. Mesa currently supports two overall kinds of spaces: grid, and continuous. Grids are divided into cells, and agents can only be on a particular cell, like pieces on a chess board. Continuous space, in contrast, allows agents to have any arbitrary position. Both grids and continuous spaces are frequently toroidal, meaning that the edges wrap around, with cells on the right edge connected to those on the left edge, and the top to the bottom. This prevents some cells having fewer neighbors than others, or agents being able to go off the edge of the environment.

Mesa has two main types of grids: SingleGrid and MultiGrid. SingleGrid enforces at most one agent per cell; MultiGrid allows multiple agents to be in the same cell. Since we want agents to be able to share a cell, we use MultiGrid.
'''
from mesa.space import MultiGrid

'''
The data collector stores three categories of data: model-level variables, agent-level variables, and tables (which are a catch-all for everything else). Model- and agent-level variables are added to the data collector along with a function for collecting them. Model-level collection functions take a model object as an input, while agent-level collection functions take an agent object as an input. Both then return a value computed from the model or each agent at their current state. When the data collector’s collect method is called, with a model object as its argument, it applies each model-level collection function to the model, and stores the results in a dictionary, associating the current value with the current step of the model. Similarly, the method applies each agent-level collection function to each agent currently in the schedule, associating the resulting value with the step of the model, and the agent’s unique_id
'''
from mesa.datacollection import DataCollector

'''
You usually won’t run a model only once, but multiple times, with fixed parameters to find the overall distributions the model generates, and with varying parameters to analyze how they drive the model’s outputs and behaviors. Instead of needing to write nested for-loops for each model, Mesa provides a BatchRunner class which automates it for you.
'''
from mesa.batchrunner import BatchRunner


'''
CanvasGrid works by looping over every cell in a grid, and generating a portrayal for every agent it finds. A portrayal is a dictionary (which can easily be turned into a JSON object) which tells the JavaScript side how to draw it. The only thing we need to provide is a function which takes an agent, and returns a portrayal object. Here’s the simplest one: it’ll draw each agent as a red, filled circle which fills half of each cell.
'''
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer


'''
The basic chart pulls data from the model’s DataCollector, and draws it as a line graph using the Charts.js JavaScript libraries. We instantiate a chart element with a list of series for the chart to track. Each series is defined in a dictionary, and has a Label (which must match the name of a model-level variable collected by the DataCollector) and a Color name. We can also give the chart the name of the DataCollector object in the model.
'''
from mesa.visualization.modules import ChartModule

In [5]:
class Consumidore(Agent):
    ''' An agent with fixed initial wealth.'''
    def __init__(self, unique_id, model, num_empresas, costo_vida, ingreso_inicial, costo_vida):
        super().__init__(unique_id, model)
        rng = default_rng()
        vals = rng.uniform()

        # Dotaciones iniciales
        self.theta = 1 / num_empresas
        self.oferta_trabajo_inicial = round(vals, 2)
        self.ingreso_inicial = ingreso_inicial
        self.oferta_trabajo = self.oferta_trabajo_inicial
        self.ingreso = self.ingreso_inicial
        self.descanso = round(default_rng().uniform(0, self.oferta_trabajo), 2)
        self.delta_oferta_trabajo = round(default_rng().uniform(0, 1 - self.oferta_trabajo), 2)

    def get_oferta_trabajo(self):
        camaradas = self.model.grid.get_cell_list_contents([self.pos])

        if len(camaradas) > 1:
            otre = self.random.choice(camaradas)

            if (otre.delta_oferta_trabajo + oferta_trabajo) <= 1:
                self.delta_oferta_trabajo = otre.delta_oferta_trabajo

    def step(self):
        self.get_oferta_trabajo()

        # Si los ingresos del consumidore son menores a los costos de vivir, muere
        if self.ingreso < costo_vida:
            self.model.grid._remove_agent(self.pos, self)
            self.model.schedule.remove(self)
        
        if self.wealth > 0:
            self.give_money()

class Empresa(Agent):
    ''' An agent with fixed initial wealth.'''
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()

class EconomiSocialista(Model):
    '''A model with some number of agents.'''
    def __init__(self, I, J, width, height):
        self.num_consumidores = I
        self.num_empresas = J
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.running = True

        # Create agents
        for i in range(self.num_consumidores):
            a = MoneyAgent(i, self)
            self.schedule.add(a)

            '''
            We instantiate a grid with width and height parameters, and a boolean as to whether the grid is toroidal. Let’s make width and height model parameters, in addition to  the number of agents, and have the grid always be toroidal. We can place agents on a grid with the grid’s place_agent method, which takes an agent and an (x, y) tuple of the coordinates to place the agent.
            '''
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        '''
        Let’s add a DataCollector to the model, and collect two variables. At the agent level, we want to collect every agent’s wealth at every step. At the model level, let’s measure the model’s Gini Coefficient, a measure of wealth inequality.
        '''
        self.datacollector = DataCollector(
            model_reporters={"Gini": compute_gini},  # `compute_gini` defined above
            agent_reporters={"Wealth": "wealth"}
        )

    def step(self):
        '''Advance the model by one step.'''

        '''
        At every step of the model, the datacollector will collect and store the model-level current Gini coefficient, as well as each agent’s wealth, associating each with the current step.
        '''
        self.datacollector.collect(self)
        self.schedule.step()

In [108]:
rng = default_rng()
vals = rng.uniform()
oferta_trabajo = 0.78
print(round(vals, 2))

print('oferta_trabajo', oferta_trabajo)
print(round(default_rng().uniform(0, 1 - oferta_trabajo), 2))

0.63
oferta_trabajo 0.78
0.1
